# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_stochpid.txt")
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02 -3.69999953e-02
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.50429671e-01 -4.92727243e-01 -5.31666025e-03]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.25610892e-01 -3.35999953e-02
 -2.42000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08749986e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90489108e-01 -5.87999953e-02
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.76979602e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48010892e-01  4.64000047e-02
 -1.04000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.83151637e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.08389108e-01  3.32000047e-02
 -2.02000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.25137655

[ 0.00000000e+00  7.95731469e+08  4.45610892e-01 -4.15999953e-02
 -1.45000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.82854667e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 51 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.36410892e-01 -3.37999953e-02
  8.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.45025378e-01  1.67203680e-01 -6.00000000e-01]
Read chunk # 52 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.02189108e-01 -2.81999953e-02
  1.33999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.85653844e-01 -2.69594451e-01  6.00000000e-01]
Read chunk # 53 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.06389108e-01 -5.79999531e-03
 -2.02000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -3.87850514e-02  6.00000000e-01]
Read chunk # 54 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.79891077e-02 -1.81999953e-02
  2.77999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-

Read chunk # 102 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.28189108e-01  9.80000469e-03
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 103 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.24010892e-01  2.32000047e-02
 -2.93000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 104 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.26989108e-01 -5.97999953e-02
 -1.10000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.62172095e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 105 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.14210892e-01  1.12000047e-02
  1.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.04351236e-02  5.92342699e-01 -6.00000000e-01]
Read chunk # 106 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.58110892e-01  3.32000047e-02
 -2.87000134e-02  0.00000000e+00 -5.3342

[ 0.00000000e+00  7.95731469e+08 -2.03889108e-01 -1.59999531e-03
  7.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.41230860e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 153 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  8.02108923e-02 -5.75999953e-02
 -1.60000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.55303446e-01  2.60448458e-01 -6.00000000e-01]
Read chunk # 154 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.38789108e-01 -2.49999953e-02
 -2.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -2.01289443e-01  6.00000000e-01]
Read chunk # 155 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.14289108e-01 -2.27999953e-02
  1.34999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.73931443e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 156 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.13710892e-01  1.34000047e-02
 -4.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.364527

[ 0.00000000e+00  7.95731469e+08 -2.21289108e-01 -7.99995308e-04
 -2.20000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.75665250e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 206 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48108923e-02  1.46000047e-02
 -2.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.28364642e-01  5.50456532e-02 -2.49661252e-01]
Read chunk # 207 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.00910892e-01  3.00000047e-02
 -1.52000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  4.65266662e-01 -6.00000000e-01]
Read chunk # 208 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.57689108e-01  2.42000047e-02
  2.37999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.83548139e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 209 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.12389108e-01 -7.19999531e-03
 -2.77000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.000000

...

[ 0.00000000e+00  7.95731469e+08  2.95108923e-02 -3.75999953e-02
  2.79999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -2.09514413e-01 -1.53278954e-01]
Read chunk # 1770 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.92910892e-01 -2.61999953e-02
  1.28999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.43643553e-01 -1.59929835e-01 -6.00000000e-01]
Read chunk # 1771 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.36108923e-02 -2.21999953e-02
  5.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.56288742e-01  3.66149890e-01 -6.00000000e-01]
Read chunk # 1772 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.35310892e-01 -2.65999953e-02
  1.60999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.82724931e-01  5.59806812e-02 -6.00000000e-01]
Read chunk # 1773 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.39010892e-01  1.82000047e-02
  2.39999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00

[ 0.00000000e+00  7.95731469e+08 -1.05189108e-01  2.30000047e-02
  2.68999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -4.86562490e-01  6.00000000e-01]
Read chunk # 1820 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.21310892e-01  1.74000047e-02
 -2.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  3.19535893e-01 -6.00000000e-01]
Read chunk # 1821 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.38110892e-01 -2.59999953e-02
 -2.58000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.53679247e-01  3.88160056e-01 -6.00000000e-01]
Read chunk # 1822 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.82289108e-01 -4.13999953e-02
  1.65999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.27271947e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1823 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.20289108e-01 -5.49999953e-02
 -1.97000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.77

[ 0.00000000e+00  7.95731469e+08 -4.25589108e-01 -4.81999953e-02
  5.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.68611830e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1869 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.25891077e-02  7.20000469e-03
  1.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.55260624e-03 -3.12380432e-01  1.80859602e-01]
Read chunk # 1870 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.43810892e-01 -4.05999953e-02
 -3.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.65373217e-01 -3.12006856e-02 -6.00000000e-01]
Read chunk # 1871 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.58589108e-01 -3.79999953e-02
 -2.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.09434744e-01 -5.42482642e-01  6.00000000e-01]
Read chunk # 1872 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.46710892e-01 -1.59999531e-03
 -2.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.67

[ 0.00000000e+00  7.95731469e+08  3.65810892e-01 -5.71999953e-02
 -1.72000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.66773768e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1919 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.12389108e-01 -3.49999953e-02
 -7.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.62987534e-02 -5.82291756e-01  6.00000000e-01]
Read chunk # 1920 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.52489108e-01 -5.61999953e-02
  6.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.84240900e-01 -1.20263578e-01  6.00000000e-01]
Read chunk # 1921 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  9.11089229e-03  8.80000469e-03
  1.04999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.30898155e-01 -2.05160369e-02 -5.33774587e-02]
Read chunk # 1922 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.32789108e-01 -1.39999531e-03
  2.30999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.98

[ 0.00000000e+00  7.95731469e+08 -3.36789108e-01 -4.41999953e-02
  1.59999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.24260526e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1970 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.13589108e-01 -1.59999953e-02
 -2.06000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.55822005e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1971 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.20189108e-01 -1.19999531e-03
 -3.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.32413842e-03 -6.00000000e-01  6.00000000e-01]
Read chunk # 1972 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.68610892e-01 -5.83999953e-02
  2.43999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1973 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  5.11108923e-02  5.94000047e-02
 -2.82000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -127.88257,
 'std': 81.13078,
 'min': -410.7058,
 'max': 0.0,
 'histogram': (array([  6,   7,  19,  21,  33,  35,  51,  58,  59,  50,  70,  93, 151,
         178, 209, 220, 273, 353, 112,   1]),
  array([-410.7058  , -390.17053 , -369.63522 , -349.09995 , -328.56464 ,
         -308.02936 , -287.49408 , -266.95877 , -246.4235  , -225.8882  ,
         -205.3529  , -184.81761 , -164.28232 , -143.74704 , -123.21175 ,
         -102.67645 ,  -82.14116 ,  -61.605873,  -41.07058 ,  -20.53529 ,
            0.      ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.099473260977188


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=20, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-17 14:40.30 [debug    ] RoundIterator is selected.
2022-04-17 14:40.30 [info     ] Directory is created at d3rlpy_logs/CQL_20220417144030
2022-04-17 14:40.30 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-17 14:40.30 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-17 14:40.30 [warning  ] Skip building models since they're already built.
2022-04-17 14:40.30 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220417144030/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/20:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 14:52.16 [info     ] CQL_20220417144030: epoch=1 step=6928 epoch=1 metrics={'time_sample_batch': 0.00029666907655174135, 'time_algorithm_update': 0.10028393924511608, 'temp_loss': 2.8203760104524895, 'temp': 0.7465105789603868, 'alpha_loss': -4.3775416721176486, 'alpha': 1.2774664221060468, 'critic_loss': 22.781028200069972, 'actor_loss': 26.226714315611275, 'time_step': 0.10086665675904403, 'td_error': 37.92656884774088, 'init_value': -71.30191040039062, 'ave_value': -63.93212064640088} step=6928
2022-04-17 14:52.16 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417144030/model_6928.pt


Epoch 2/20:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 15:02.42 [info     ] CQL_20220417144030: epoch=2 step=13856 epoch=2 metrics={'time_sample_batch': 0.00029326410959829637, 'time_algorithm_update': 0.08866798440661221, 'temp_loss': 0.9077620992768237, 'temp': 0.4195573395569798, 'alpha_loss': 0.6452432500244717, 'alpha': 1.2477544136775558, 'critic_loss': 113.6042675585174, 'actor_loss': 92.3157616746343, 'time_step': 0.08927167986474467, 'td_error': 80.32413035570184, 'init_value': -130.15475463867188, 'ave_value': -120.32872105798826} step=13856
2022-04-17 15:02.42 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417144030/model_13856.pt


...

Epoch 19/20:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 18:08.24 [info     ] CQL_20220417144030: epoch=19 step=131632 epoch=19 metrics={'time_sample_batch': 0.00033422361620579253, 'time_algorithm_update': 0.09361962623051077, 'temp_loss': 0.0010802490694779585, 'temp': 0.15126606310324153, 'alpha_loss': 0.027971341859239756, 'alpha': 0.3347467774908138, 'critic_loss': 38.89104494267502, 'actor_loss': 19.261070990225186, 'time_step': 0.09429922761873065, 'td_error': 18.72637916507333, 'init_value': -22.360034942626953, 'ave_value': -20.8096111468993} step=131632
2022-04-17 18:08.24 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417144030/model_131632.pt


Epoch 20/20:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-17 18:19.17 [info     ] CQL_20220417144030: epoch=20 step=138560 epoch=20 metrics={'time_sample_batch': 0.0003328250751209039, 'time_algorithm_update': 0.09206571142612924, 'temp_loss': 0.00038696179106884244, 'temp': 0.14947076501673212, 'alpha_loss': 0.026300459506297277, 'alpha': 0.3118550264239999, 'critic_loss': 38.48978160570352, 'actor_loss': 18.019496157627735, 'time_step': 0.09274033637713064, 'td_error': 18.26813940368282, 'init_value': -20.709875106811523, 'ave_value': -19.23658484142155} step=138560
2022-04-17 18:19.17 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417144030/model_138560.pt


[(1,
  {'time_sample_batch': 0.00029666907655174135,
   'time_algorithm_update': 0.10028393924511608,
   'temp_loss': 2.8203760104524895,
   'temp': 0.7465105789603868,
   'alpha_loss': -4.3775416721176486,
   'alpha': 1.2774664221060468,
   'critic_loss': 22.781028200069972,
   'actor_loss': 26.226714315611275,
   'time_step': 0.10086665675904403,
   'td_error': 37.92656884774088,
   'init_value': -71.30191040039062,
   'ave_value': -63.93212064640088}),
 (2,
  {'time_sample_batch': 0.00029326410959829637,
   'time_algorithm_update': 0.08866798440661221,
   'temp_loss': 0.9077620992768237,
   'temp': 0.4195573395569798,
   'alpha_loss': 0.6452432500244717,
   'alpha': 1.2477544136775558,
   'critic_loss': 113.6042675585174,
   'actor_loss': 92.3157616746343,
   'time_step': 0.08927167986474467,
   'td_error': 80.32413035570184,
   'init_value': -130.15475463867188,
   'ave_value': -120.32872105798826}),
 (3,
  {'time_sample_batch': 0.00033041993134566875,
   'time_algorithm_update': 0

In [10]:
model.save_model('cqlStochpid2000_Ep20.pt')
model.save_policy('cqlStochpid2000_Ep20.pt')

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

start
[ 0.00000000e+00  7.95731469e+08 -2.00289108e-01  1.20000047e-02
 -1.88000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.04329141e-01  1.07235845e-01  6.00000000e-01]
Read chunk # 2001 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.21689108e-01 -4.79999953e-02
  2.36999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 2002 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.75789108e-01 -3.21999953e-02
  1.39999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78630064e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 2003 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.79910892e-01 -3.15999953e-02
  3.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.39455184e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2004 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.31489108e-01  2.30000047e-02
 -4.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


[ 0.00000000e+00  7.95731469e+08 -9.31891077e-02 -3.55999953e-02
 -2.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.63507822e-03  6.73487384e-02  6.00000000e-01]
Read chunk # 2046 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.00389108e-01 -3.65999953e-02
 -1.90000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.28812898e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 2047 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.33910892e-01  1.88000047e-02
  9.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.09151976e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2048 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.47410892e-01 -5.99995308e-04
  2.57999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.06779283e-02 -6.00000000e-01]
Read chunk # 2049 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  8.83108923e-02  2.48000047e-02
 -2.27000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.38

[ 0.00000000e+00  7.95731469e+08  3.55010892e-01  3.20000047e-02
 -1.36000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.53899655e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2099 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.40489108e-01 -5.19999953e-02
 -2.57000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -3.24939354e-01  6.00000000e-01]
Read chunk # 2100 out of 4999
torch.Size([111000, 6])
2022-04-17 18:19.17 [debug    ] RoundIterator is selected.
2022-04-17 18:19.17 [info     ] Directory is created at d3rlpy_logs/FQE_20220417181917
2022-04-17 18:19.17 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-17 18:19.17 [debug    ] Building models...
2022-04-17 18:19.17 [debug    ] Models have been built.
2022-04-17 18:19.17 [info     ] Parameters are saved to d3rlpy_logs/FQE_20220417181917/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'batch_s

Epoch 1/100:   0%|          | 0/876 [00:00<?, ?it/s]

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

2022-04-17 18:25.23 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417181917/model_86724.pt


Epoch 100/100:   0%|          | 0/876 [00:00<?, ?it/s]

2022-04-17 18:25.27 [info     ] FQE_20220417181917: epoch=100 step=87600 epoch=100 metrics={'time_sample_batch': 0.00016191887528928993, 'time_algorithm_update': 0.00403139934147874, 'loss': 0.19798493267544098, 'time_step': 0.004266368199701179, 'init_value': -11.891968727111816, 'ave_value': -10.092297953292055, 'soft_opc': nan} step=87600


2022-04-17 18:25.27 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417181917/model_87600.pt


[(1,
  {'time_sample_batch': 0.0001437021717088952,
   'time_algorithm_update': 0.0030731534304684156,
   'loss': 0.007757241122975316,
   'time_step': 0.0032807988663242286,
   'init_value': -1.0139727592468262,
   'ave_value': -0.7032107809085298,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00014131580857925762,
   'time_algorithm_update': 0.003101518437198308,
   'loss': 0.0060979163775977474,
   'time_step': 0.003304141841522635,
   'init_value': -1.4636434316635132,
   'ave_value': -1.0110418039432785,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00013856392472846323,
   'time_algorithm_update': 0.00310577458986953,
   'loss': 0.007260493202970605,
   'time_step': 0.0033101047555061237,
   'init_value': -1.9516700506210327,
   'ave_value': -1.4125365340394211,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.0001628491432155104,
   'time_algorithm_update': 0.003677782947069978,
   'loss': 0.010123744105827701,
   'time_step': 0.003917415664620595,
   'init_val

In [12]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [13]:
from d3rlpy.torch_utility import to_cpu
to_cpu(model)
model.save_policy("cqlStochpid2000Ep20CPU.pt")
model.save_model("cqlStochpid2000Ep20modelCPU.pt")

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(
